## Atividades Desenvolvidas no Notebook - 00_configuracao_ambiente

|Sequência|Ação|Detalhamento
|---|---|---|
|SEQ-01|Configuração do ambiente no Databricks|Criação do storage "dbfs:/FileStore/tables/landing_zone"|
|SEQ-02 / SEQ-03 / SEQ-04|Criação de Schemas (Bancos de Dados)|Criação das Camadas Bronze, Silver e Gold |
|SEQ-05 / SEQ-06 / SEQ-07|Criação das Tabelas|Criação das Tabelas das Camadas Bronze, Silver e Gold|
|SEQ-08|Criação Script Dimensão Tempo|Geração de Scripts de Insert para a Dimensão Tempo para o mês de Jul/2023|

In [0]:
from pyspark.sql.functions import *
from datetime import date, timedelta


In [0]:
%run /estudos/libs/bibliotecas

In [0]:
diretorio_landing_zone = "dbfs:/FileStore/tables/landing_zone"

try:
    dbutils.fs.ls(diretorio_landing_zone)
    dbutils.fs.rm(diretorio_landing_zone, True)
        
except:

    print(f"O diretório {diretorio_landing_zone} não existe.")


In [0]:
try:

    dbutils.fs.ls(diretorio_landing_zone)
        
except:

    print(f"O diretório {diretorio_landing_zone} não existe e será criado.")
    dbutils.fs.mkdirs(diretorio_landing_zone)

O diretório dbfs:/FileStore/tables/landing_zone não existe e será criado.


Out[6]: <function __main__.acentuacao(palavra)>

In [0]:
%sql
DROP SCHEMA IF EXISTS bronze CASCADE;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS bronze COMMENT 'SCHEMA (Banco de Dados) referente a Camada BRONZE.';

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/tb_forma_pagamento', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/tb_categoria_produto', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/tb_produto', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/tb_cliente', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/tb_vendedor', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/tb_loja', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/tb_venda', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/bronze.db/tb_item_venda', True);

In [0]:
%sql
DROP SCHEMA IF EXISTS silver CASCADE;

In [0]:
%sql 
CREATE SCHEMA IF NOT EXISTS silver COMMENT 'SCHEMA (Banco de Dados) referente a Camada SILVER.';

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/tb_forma_pagamento', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/tb_categoria_produto', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/tb_produto', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/tb_cliente', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/tb_vendedor', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/tb_loja', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/tb_venda', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/silver.db/tb_item_venda', True);

In [0]:
%sql
DROP SCHEMA IF EXISTS gold CASCADE;

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS gold COMMENT 'SCHEMA (Banco de Dados) referente a Camada GOLD.';

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/dim_forma_pagamento', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/dim_categoria_produto', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/dim_produto', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/dim_cliente', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/dim_vendedor', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/dim_loja', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/fato_venda', True);
dbutils.fs.rm('dbfs:/user/hive/warehouse/gold.db/dim_tempo', True);

In [0]:
%sql

DROP  TABLE IF EXISTS bronze.tb_forma_pagamento;
CREATE TABLE bronze.tb_forma_pagamento
(
CODIGO_FORMA_PAGAMENTO	 INT COMMENT 'Código da Forma de Pagamento na Tabela TB_FORMA_PAGAMENTO.'
,DESCRICAO_FORMA_PAGAMENTO VARCHAR(100) COMMENT 'Descrição da Forma de Pagamento na Tabela TB_FORMA_PAGAMENTO.'
)
COMMENT 'Armazena informações de forma de pagamento.';

DROP  TABLE IF EXISTS bronze.tb_categoria_produto;
CREATE TABLE bronze.tb_categoria_produto
(
CODIGO_CATEGORIA INT COMMENT 'Código da Categoria do Produto.'
,CATEGORIA_PRODUTO VARCHAR(100) COMMENT 'Descrição da Categoria do Produto.'
,SUB_CATEGORIA_PRODUTO VARCHAR(100) COMMENT ''

)
COMMENT 'Armazena informações das categorias dos produtos';

DROP  TABLE IF EXISTS bronze.tb_produto;
CREATE TABLE bronze.tb_produto
(
CODIGO_PRODUTO	 INT  COMMENT 'Código do Produto.'
,DESCRICAO_PRODUTO	 VARCHAR(100) COMMENT 'Descrição do Produto.'
,CODIGO_CATEGORIA INT COMMENT 'Categoria do Produto.'
)
COMMENT '	Armazena informações de produtos.';

DROP  TABLE IF EXISTS bronze.tb_cliente;
CREATE TABLE bronze.tb_cliente
(
CODIGO_CLIENTE int  COMMENT 'Código do Cliente.',
NOME_CLIENTE varchar(100) COMMENT 'Nome do Cliente.',
DATA_NASCIMENTO date,
IDADE_CLIENTE int,
DESCRICAO_ENDERECO varchar(200),
NUMERO_ENDERECO int,
NOME_CIDADE varchar(100),
NUMERO_TELEFONE varchar(30),
EMAIL_CLIENTE varchar(100),
OCUPACAO_CLIENTE varchar(100)
)
COMMENT 'Armazena informações de clientes.';

DROP  TABLE IF EXISTS bronze.tb_vendedor;
CREATE TABLE bronze.tb_vendedor
(
CODIGO_VENDEDOR INT COMMENT 'Código do Vendedor.'
,NOME_VENDEDOR VARCHAR(100) COMMENT 'Nome do Vendedor.'
)
COMMENT 'Armazena informações de vendedores.';

DROP  TABLE IF EXISTS bronze.tb_loja;
CREATE TABLE bronze.tb_loja
(
CODIGO_LOJA_VENDEDOR INT,
CODIGO_LOJA INT COMMENT 'Código da Loja.',
CODIGO_VENDEDOR INT,
NOME_LOJA varchar(100) COMMENT 'Nome da Loja.',
LOCALIDADE_LOJA varchar(100) COMMENT 'Localidade da Loja.',
TIPO_LOJA varchar(100) COMMENT 'Tipo da Loja.'
)
COMMENT 'Armazena informações de lojas.';

DROP  TABLE IF EXISTS bronze.tb_venda;
CREATE TABLE bronze.tb_venda
(
CODIGO_VENDA INT COMMENT 'Código da Venda.'
,DATA_VENDA	TIMESTAMP COMMENT 'Data da Venda'
,CODIGO_CLIENTE	INT COMMENT 'Código do Cliente.'
,CODIGO_LOJA_VENDEDOR	INT COMMENT 'Código da Loja.'
,VALOR_FINAL DECIMAL(18,2)	 COMMENT 'Valor Total da Venda.'
,CODIGO_FORMA_PAGAMENTO INT COMMENT 'Forma de Pagamento.'
,TIPO_PAGAMENTO VARCHAR(100) COMMENT 'Tipo de Pagamento.'
)
COMMENT 'Armazenada informações de vendas.';

DROP  TABLE IF EXISTS bronze.tb_item_venda;
CREATE TABLE bronze.tb_item_venda
(
CODIGO_VENDA INT COMMENT 'Código da Venda.'
,CODIGO_PRODUTO	INT COMMENT 'Código do Produto.'
,VALOR_UNITARIO DECIMAL(18,2)	 COMMENT 'Valor Unitário do Produto.'
,QUANTIDADE	INT COMMENT 'Quantidade do Produto Vendido.'
,VALOR_FINAL DECIMAL(18,2) COMMENT 'Valor Final do Produto Vendido.'
)
COMMENT 'Armazena informações sobre os itens que foram vendidos.';

In [0]:
%sql

DROP  TABLE IF EXISTS silver.tb_forma_pagamento;
CREATE TABLE silver.tb_forma_pagamento
(
CODIGO_FORMA_PAGAMENTO	 INT COMMENT 'Código da Forma de Pagamento na Tabela TB_FORMA_PAGAMENTO.'
,DESCRICAO_FORMA_PAGAMENTO VARCHAR(100) COMMENT 'Descrição da Forma de Pagamento na Tabela TB_FORMA_PAGAMENTO.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
)
COMMENT 'Armazena informações de forma de pagamento.';

DROP  TABLE IF EXISTS silver.tb_categoria_produto;
CREATE TABLE silver.tb_categoria_produto
(
CODIGO_CATEGORIA INT COMMENT 'Código da Categoria do Produto.'
,CATEGORIA_PRODUTO VARCHAR(100) COMMENT 'Descrição da Categoria do Produto.'
,SUB_CATEGORIA_PRODUTO VARCHAR(100) COMMENT ''
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
)
COMMENT 'Armazena informações das categorias dos produtos';

DROP  TABLE IF EXISTS silver.tb_produto;
CREATE TABLE silver.tb_produto
(
CODIGO_PRODUTO	 INT  COMMENT 'Código do Produto.'
,DESCRICAO_PRODUTO	 VARCHAR(100) COMMENT 'Descrição do Produto.'
,CODIGO_CATEGORIA INT COMMENT 'Categoria do Produto.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
)
COMMENT '	Armazena informações de produtos.';

DROP  TABLE IF EXISTS silver.tb_cliente;
CREATE TABLE silver.tb_cliente
(
CODIGO_CLIENTE INT COMMENT 'Código do Cliente.'
,NOME_CLIENTE VARCHAR(100) COMMENT 'Nome do Cliente.'
,DATA_NASCIMENTO DATE
,IDADE_CLIENTE INT
,DESCRICAO_ENDERECO VARCHAR(200)
,NUMERO_ENDERECO INT
,NOME_CIDADE VARCHAR(100)
,NUMERO_TELEFONE VARCHAR(30)
,EMAIL_CLIENTE VARCHAR(100)
,OCUPACAO_CLIENTE VARCHAR(100)
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
)
COMMENT 'Armazena informações de clientes.';

DROP  TABLE IF EXISTS silver.tb_vendedor;
CREATE TABLE silver.tb_vendedor
(
CODIGO_VENDEDOR INT COMMENT 'Código do Vendedor.'
,NOME_VENDEDOR VARCHAR(100) COMMENT 'Nome do Vendedor.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
)
COMMENT 'Armazena informações de vendedores.';

DROP  TABLE IF EXISTS silver.tb_loja;
CREATE TABLE silver.tb_loja
(
CODIGO_LOJA_VENDEDOR INT
,CODIGO_LOJA INT COMMENT 'Código da Loja.'
,CODIGO_VENDEDOR INT
,NOME_LOJA varchar(100) COMMENT 'Nome da Loja.'
,LOCALIDADE_LOJA varchar(100) COMMENT 'Localidade da Loja.'
,TIPO_LOJA varchar(100) COMMENT 'Tipo da Loja.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
)
COMMENT 'Armazena informações de lojas.';

DROP  TABLE IF EXISTS silver.tb_venda;
CREATE TABLE silver.tb_venda
(
CODIGO_VENDA INT COMMENT 'Código da Venda.'
,DATA_VENDA	TIMESTAMP COMMENT 'Data da Venda'
,CODIGO_CLIENTE	INT COMMENT 'Código do Cliente.'
,CODIGO_LOJA_VENDEDOR	INT COMMENT 'Código do Vendedor.'
,VALOR_FINAL DECIMAL(18,2)	 COMMENT 'Valor Total da Venda.'
,CODIGO_FORMA_PAGAMENTO	INT COMMENT 'Forma de Pagamento.'
,TIPO_PAGAMENTO VARCHAR(100) COMMENT 'Tipo de Pagamento.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
)
COMMENT 'Armazenada informações de vendas.';

DROP  TABLE IF EXISTS silver.tb_item_venda;
CREATE TABLE silver.tb_item_venda
(
CODIGO_VENDA INT COMMENT 'Código da Venda.'
,CODIGO_PRODUTO	INT COMMENT 'Código do Produto.'
,VALOR_UNITARIO DECIMAL(18,2)	 COMMENT 'Valor Unitário do Produto.'
,QUANTIDADE	INT COMMENT 'Quantidade do Produto Vendido.'
,VALOR_FINAL DECIMAL(18,2) COMMENT 'Valor Final do Produto Vendido.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
)
COMMENT 'Armazena informações sobre os itens que foram vendidos.';

In [0]:
%sql

DROP  TABLE IF EXISTS gold.dim_forma_pagamento;
CREATE TABLE gold.dim_forma_pagamento
(
ID_FORMA_PAGAMENTO VARCHAR(100) COMMENT 'Chave da Dimensão Forma de Pagamento.'
,CODIGO_FORMA_PAGAMENTO	 INT COMMENT 'Código da Forma de Pagamento na Tabela TB_FORMA_PAGAMENTO.'
,DESCRICAO_FORMA_PAGAMENTO VARCHAR(100) COMMENT 'Descrição da Forma de Pagamento na Tabela TB_FORMA_PAGAMENTO.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
);

DROP  TABLE IF EXISTS gold.dim_produto;
CREATE TABLE gold.dim_produto
(
ID_PRODUTO VARCHAR(100) COMMENT 'Chave da Dimensão Produto.'
,CODIGO_PRODUTO	 INT  COMMENT 'Código do Produto.'
,DESCRICAO_PRODUTO VARCHAR(100) COMMENT 'Descrição do Produto.'
,CATEGORIA_PRODUTO VARCHAR(100) COMMENT 'Categoria do Produto.'
,SUB_CATEGORIA_PRODUTO VARCHAR(100) COMMENT 'Categoria do Produto.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
);

DROP  TABLE IF EXISTS gold.dim_cliente;
CREATE TABLE gold.dim_cliente
(
ID_CLIENTE VARCHAR(100) COMMENT 'Chave da Dimensão Cliente.'
,CODIGO_CLIENTE INT COMMENT 'Código do Cliente.'
,NOME_CLIENTE VARCHAR(100) COMMENT 'Nome do Cliente.'
,DATA_NASCIMENTO DATE COMMENT 'Data de Nascimento do Cliente'
,IDADE_CLIENTE INT COMMENT 'Idade do Cliente'
,DESCRICAO_ENDERECO VARCHAR(200) COMMENT 'Endereço do Cliente'
,NUMERO_ENDERECO INT COMMENT 'Numero da Residencia do Cliente'
,NOME_CIDADE VARCHAR(100) COMMENT 'Cidade de Residência do Cliente'
,NUMERO_TELEFONE VARCHAR(30) COMMENT 'Telefone do Cliente'
,EMAIL_CLIENTE VARCHAR(100) COMMENT 'E-mail do Cliente'
,OCUPACAO_CLIENTE VARCHAR(100) COMMENT 'Ocupação do cliente'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
);

DROP  TABLE IF EXISTS gold.dim_loja;
CREATE TABLE gold.dim_loja
(
ID_LOJA VARCHAR(100) COMMENT 'Chave da Dimensão Loja.'
,CODIGO_LOJA	 INT COMMENT 'Código da Loja.'
,NOME_LOJA	 VARCHAR(100) COMMENT 'Nome da Loja.'
,NOME_VENDEDOR VARCHAR(100)  COMMENT 'Nome do Vendedor.'
,LOCALIDADE_LOJA	 VARCHAR(100) COMMENT 'Localidade da Loja.'
,TIPO_LOJA VARCHAR(100) COMMENT 'Tipo da Loja.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
);

DROP  TABLE IF EXISTS gold.dim_tempo;
CREATE TABLE gold.dim_tempo
(
ID_TEMPO VARCHAR(100) COMMENT 'Chave da Dimensão Tempo.'
,DATA VARCHAR(10) COMMENT 'Data.'
,ANO INT COMMENT 'Ano referente a coluna data.'
,MES	INT COMMENT 'Mês referente a coluna data.'
,DIA INT COMMENT 'Dia referente a coluna data.' 
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
);

DROP  TABLE IF EXISTS gold.fato_venda;
CREATE TABLE gold.fato_venda
(
CODIGO_VENDA INT COMMENT 'Código da Venda.'
,ID_CLIENTE	VARCHAR(100)  COMMENT 'Chave da Dimensão Cliente.'
,ID_LOJA VARCHAR(100) COMMENT 'Chave da Dimensão Loja.'
,ID_PRODUTO	VARCHAR(100)  COMMENT 'Chave da Dimensão Produto.'
,ID_FORMA_PAGAMENTO	 VARCHAR(100) COMMENT 'Chave da Dimensão Forma de Pagamento.'
,ID_TEMPO VARCHAR(100)  COMMENT 'Chave da Dimensão Tempo.'
,HORA_VENDA	INT COMMENT 'Hora em que ocorreu a venda.'
,QUANTIDADE	INT COMMENT 'Quantidade vendida do produto.'
,VALOR_UNITARIO DECIMAL(18,2)	 COMMENT 'Valor unitário do produto.'
,VALOR_FINAL DECIMAL(18,2)	 COMMENT 'Valor vendido de um produto.'
,TIPO_PAGAMENTO VARCHAR(100) COMMENT 'Tipo de Pagamentto da Venda.'
,DATA_CADASTRO TIMESTAMP COMMENT 'Data de Cadastro do Registro.'
,DATA_ALTERACAO TIMESTAMP COMMENT 'Data de Atualização do Registro.'
);

In [0]:
data_inicial = date(2017, 1, 1)
data_fim = date(2017, 1, 31)
dias_acrescimo = timedelta(days=1)

cria_dimensao_tempo(data_inicial, data_fim, dias_acrescimo)

In [0]:
%sql

SELECT * FROM gold.DIM_TEMPO

ID_TEMPO,DATA,ANO,MES,DIA,DATA_CADASTRO,DATA_ALTERACAO
cb53f1ea42eb3ad7855982c4412e5212ea4b55c33860ac31aeee1f4a5a503e56,20170127,2017,1,27,2024-02-11T09:49:06.384+0000,2024-02-11T09:49:06.384+0000
29a08187ffdd05dd353d22eedcdd295d0fdf7437e28a739de9b3f9a7dc2026ac,20170129,2017,1,29,2024-02-11T09:49:15.163+0000,2024-02-11T09:49:15.163+0000
7dfe274deab5101df0def9362763c079f9b02bec1c151cd3897dbe1245a3b8f2,20170104,2017,1,4,2024-02-11T09:46:54.518+0000,2024-02-11T09:46:54.518+0000
a698733cddcb06343d75c33a8e17760e3ecb54fa02c283660d16718d78b22bd3,20170116,2017,1,16,2024-02-11T09:48:09.679+0000,2024-02-11T09:48:09.680+0000
6e684b0511395c88f2b5c349d9e70420796ad29d6e2e40fd8fdafa3a01f8a662,20170112,2017,1,12,2024-02-11T09:47:46.454+0000,2024-02-11T09:47:46.454+0000
96ee42494fc286d2393db6166d70820066c9c7dc3df77540554c1310bf1c4b28,20170122,2017,1,22,2024-02-11T09:48:44.250+0000,2024-02-11T09:48:44.250+0000
db586a70bfd24764545d3278bcbb89e72958ea38430ab0181dc2301de1402a26,20170118,2017,1,18,2024-02-11T09:48:19.753+0000,2024-02-11T09:48:19.753+0000
c3b68bfe8ac66110140fec08618f58b2c95aed1180f024be9ad3b75b44c1c3b8,20170123,2017,1,23,2024-02-11T09:48:48.256+0000,2024-02-11T09:48:48.256+0000
3b014864f67350297da4fe23909fa0330a3e585020cd34ec072110b8cfd3d22b,20170105,2017,1,5,2024-02-11T09:46:57.755+0000,2024-02-11T09:46:57.755+0000
2ebdb7659408fa2fabea151a512149cb1c9a6a801ff9d31c54aa6506830ac9a5,20170106,2017,1,6,2024-02-11T09:47:03.765+0000,2024-02-11T09:47:03.765+0000


In [0]:
%sql
SHOW TABLES FROM silver

In [0]:
%sql
DESC gold.DIM_LOJA

In [0]:
%sql
DESC DETAIL gold.DIM_LOJA

In [0]:
%sql
DESC SCHEMA bronze